In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [29]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException

# def run_sel_lazyloading(driver):
#     # Get the current number of rows
#     current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
#     total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('wines')[0])
#     while current_rows_number < total_number:
#         # Scroll down to make new XHR (request more table rows)
#         driver.find_element_by_tag_name('body').send_keys(Keys.END)
#         try:
#             driver.set_window_position(0,-1000)
#             # Wait until number of rows increased       
#             wait(driver, 60).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
#             # Update variable with current rows number
#             current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
#             print(current_rows_number)
#         # If number of rows remains the same after 5 seconds passed, break the loop
#         # as there no more rows to receive
#         except TimeoutException:
#             break

def run_sel_lazyloading_test(driver):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)

In [53]:
import requests

# test_wine_url = "https://www.vivino.com/quady-winery-electra-red-california-moscato/w/14175?year=2016&price_id=14978727"
# r = requests.get(test_wine_url, headers={'user-agent': 'Mozilla/5.0'})

# soup = BeautifulSoup(r.content, "html")
# print(soup)
# price_div = soup.find("span", {"class": "purchaseAvailability__currentPrice--3mO4u"})
# print(price_div)

def get_variables(url):
    price = 0.0
    vintage = ""
    review = ""
    test_wine_url = url
    test_wine = WineSetup(test_wine_url)
    test_wine.setup_driver()
    run_sel_lazyloading_test(test_wine.driver)

    test_html = BeautifulSoup(test_wine.driver.page_source, 'lxml')
    price_div = test_html.find("span", {"class": "purchaseAvailability__currentPrice--3mO4u"})
    vintage_div = test_html.find("span", {"class": "purchaseAvailability__year--3huqD"})
    review_div = test_html.find("div", {"class": "communityReview__textSection--vu-i-"})
    if price_div:
        price = float(price_div.text.split("$")[1])
    if vintage_div:
        vintage = vintage_div.text
    if review_div:
        review = review_div.find("p").text
    
    return price, vintage, review


In [54]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

test = wine_db.wines.find({'price': 0.0, 'type': 'Port'})
count = 0
for t in test:
    count += 1
    price, vintage, review = get_variables(t['url'])
    
    print('-----------------')
    print('Price: ', price)
    print('Vintage: ', vintage)
    print('Review: ', review)
    
print(count)

AttributeError: 'NoneType' object has no attribute 'text'

In [46]:
wine_db.wines.update({'url': test_wine_url}, {"$set": {"price": price, "vintage": vintage, "reviews": review}})
# #launch url
# austria_wine_url = "https://www.vivino.com/explore?e=eJwNyzEKgDAMRuHbZK6D4795A3ESkRhrKdhWklL09nZ5y8dLioFSzHCU-MXoHMmHZSbpmejpGi401ugr31QOKNeYg-3cvHLwVHB6k37ZuoErGYR_v5AdOA=="

# # # create a new Firefox session
# path = r'/Users/yuewengmak/Desktop/chromedriver'
# driver = webdriver.Chrome(executable_path = path)
# driver.implicitly_wait(30)
# driver.get(austria_wine_url)

# tracking(driver)

/Users/yuewengmak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}